In [1]:
# import pandas as pd
# import numpy as np
# import warnings


# pd.options.mode.chained_assignment = None
# pd.set_option('display.max_columns', 80)
# pd.set_option('display.max_rows', 55)

# warnings.simplefilter(action='ignore', category=FutureWarning)

# item_master_excel_file = '../data/inzi/Item Master_08_18_2021 0317.xls'
# material_history_excel_file = '../data/inzi/Material Movement History_08_19_2021 1026.xls'
# out_put_file = '../output/9-15.xlsx'

# item_master = pd.read_excel(
#     item_master_excel_file, index_col=False, header=1
# )
# material_history = pd.read_excel(
#     material_history_excel_file, index_col=False, header=1
# )

# # Item master excel file must have 3 columns 'Material', 'Material Type', 'Procurement'
# temp_item_master = item_master[['Material', 'Material Type', 'Procurement']]

# '''
# PREPARE FOR STEP ONE
# '''
# step_one = pd.merge(material_history, temp_item_master,
#                     on='Material', how='left')

# step_one.rename(columns={
#     'Reference': 'Order Category',
#     'Unnamed: 13': 'Order Data',
#     'Unnamed: 14': 'Master Category',
#     'Unnamed: 15': 'Master Data',
#     'Unnamed: 16': 'Remark'
# }, inplace=True)

# step_one = step_one.iloc[1:, :]

# # pd.set_option('mode.chained_assignment', None)

# step_one.loc[step_one['Quantity'] > 0, 'Input'] = step_one['Quantity']
# step_one.loc[step_one['Quantity'] > 0, 'Output'] = 0
# step_one.loc[step_one['Quantity'] <= 0, 'Input'] = 0
# step_one.loc[step_one['Quantity'] <= 0, 'Output'] = -step_one['Quantity']

# cols = step_one.columns.tolist()

# cols = cols[:10] + [cols[-2], cols[-1]] + cols[10:-2]

# step_one = step_one[cols]

# step_one['Account code'] = [int(x)
#                             for x in (step_one['Movement Type'].str[:3])]

# cols = step_one.columns.tolist()
# cols = cols[:7] + [cols[-1]] + cols[7: -1]
# step_one = step_one[cols]


In [2]:
# steady = step_one.pivot_table(index=['Unnamed: 0', 'Material'],
#                             columns='Account code', values=['Quantity'], aggfunc=np.sum, fill_value=0)

# steady.reset_index(inplace=True)
# del steady['Unnamed: 0']
# steady = steady.groupby(steady['Material']).aggregate(
#         'sum')

# steady['IN'] = steady.where(steady > 0).sum(1)
# steady['OUT'] = steady.where(steady < 0).sum(1)

# steady_2 = steady.iloc[:, [-2, -1]]


In [3]:
# from functools import reduce

# def merge_result(dfs):
#     df_final = reduce(lambda left,right: pd.merge(left, right, how="outer", on=['Material'], sort=False,
#                     indicator=False, validate=None, suffixes=('_01', '_02',)), 
#                     dfs)
#     return df_final

# stock = pd.read_excel('../data/inzi/stock.xlsb', header=2)
# stock.rename(columns={
#     'Unnamed: 5': 'Materials'
# }, inplace=True)
# stock['OPENING STOCK'] = stock['Unrestricted'] + stock['Inspection'] + stock['Returned'] + stock['Blocked']
# stock = stock[['Materials', 'OPENING STOCK']]
# stock.rename(columns={'Materials': 'Material'}, inplace=True)

# temp = pd.merge(stock, aug_28, how="outer", on=['Material'], sort=False,
#                     indicator=False, validate=None)



# temp_aug = pd.merge(aug_28, stock, how="outer", on=['Material'], sort=False,
#                     indicator=False, validate=None)

# temp_aug

# temp_aug.set_axis([*temp_aug.columns[:-1], 'temporery'], axis=1, inplace=True)

# temp_aug['CLOSING_STOCK'] = temp_aug[[101, 102, 'temporery']].sum(axis=1) - temp_aug[[401, 720]].sum(axis=1)

# temp_aug[temp_aug.columns[[0, -1]]]

# dfs = [aug_28, aug_29, aug_28, aug_29, aug_28, aug_29, aug_28, aug_29, aug_28]
# result = pd.concat(dfs)
# result['temp'] = 0

# result = result[['Material', 'temp']]

# result.drop_duplicates(inplace=True)

# final_result = pd.concat(dfs)
# final_result['temp'] = 0
# final_result = final_result[['Material', 'temp']]
# final_result.drop_duplicates(inplace=True)

# final_result = pd.merge(final_result, stock[stock.columns[[0, -1]]], how="left", on=['Material'], sort=False,
#                         indicator=False, validate=None)

In [4]:
# import numpy as np
# from functools import reduce
import warnings

import pandas as pd

pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 80)
pd.set_option('display.max_rows', 55)

warnings.simplefilter(action='ignore', category=FutureWarning)

open_stock = '../data/inzi/stock.xlsb'

files = [
    '../data/inzi/02 SEP/01-09-2021_IN-OUT DAILY REPORT.xlsb',
    '../data/inzi/03 SEP/02-09-2021_IN-OUT DAILY REPORT.xlsb',
    '../data/inzi/04 SEP/03-09-2021_IN-OUT DAILY REPORT.xlsb',
]

out_put_file = '../output/concet.xlsx'


stock = pd.read_excel(open_stock, header=2)

stock.rename(columns={
    'Unnamed: 5': 'Materials'
}, inplace=True)
stock['OPENING STOCK'] = stock['Unrestricted'] + \
    stock['Inspection'] + stock['Returned'] + stock['Blocked']
stock = stock[['Materials', 'OPENING STOCK']]
stock.rename(columns={'Materials': 'Material'}, inplace=True)
stock = stock.groupby(stock['Material']).aggregate(
    'sum')


def get_concat(sheet):
    dfs = []
    for file_name in files:
        df = pd.read_excel(file_name, sheet_name=sheet, header=8)
        df.drop(columns=['Unnamed: 0', 'Unnamed: 2',
                         'IN', 'OUT'], inplace=True)
        df.rename(columns={'Unnamed: 1': 'Material'}, inplace=True)
        if 'Unnamed: 5' in df:
            df.rename(columns={'Unnamed: 5': 'PURCHASE'}, inplace=True)
        dfs.append(df)

    final_result = pd.concat(dfs)
    final_result['temp'] = 0
    final_result = final_result[['Material', 'temp']]
    final_result.drop_duplicates(inplace=True)
    final_result = pd.merge(final_result, stock, how="left", on=['Material'], sort=False,
                            indicator=False, validate=None)
    del final_result['temp']
    l = len(dfs)

    for _ in range(l):
        in_stock = [344, 401, 623, 720, 321, 323, 327, 602]
        sum_col = []
        out_stock = [401, 720]
        subtract_col = []

        dfs[0] = pd.merge(dfs[0], final_result[final_result.columns[[0, -1]]], how="outer", on=['Material'], sort=False,
                          indicator=False, validate=None)
        dfs[0].set_axis([*dfs[0].columns[:-1], 'temporary'],
                        axis=1, inplace=True)

        for column in in_stock:
            if column in dfs[0].columns:
                sum_col.append(column)
        sum_col.append('temporary')

        for column in out_stock:
            if column in dfs[0].columns:
                subtract_col.append(column)

        dfs[0]['CLOSING STOCK'] = dfs[0][sum_col].sum(
            axis=1) - dfs[0][subtract_col].sum(axis=1)
        del dfs[0]['temporary']

        final_result = pd.merge(final_result, dfs[0], how="outer", on=['Material'], sort=False,
                                indicator=False, validate=None)
        del dfs[0]
    print(f'Concat {l} files together in {sheet}')
    # final_result.drop(final_result.columns[-1], axis=1, inplace=True)
    final_result.fillna(0, inplace=True)
    return final_result


raw_concat = get_concat('REPORT-RAW')
wip_concat = get_concat('REPORT-WIP')
fg_concat = get_concat('REPORT-FG')

Concat 3 files together in REPORT-RAW
Concat 3 files together in REPORT-WIP
Concat 3 files together in REPORT-FG


In [21]:
import warnings

import pandas as pd

pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 80)
pd.set_option('display.max_rows', 55)



previous = pd.read_excel('../data/inzi/08 SEP/07-09-2021_IN-OUT-STOCK_DAILY REPORT.xlsb', sheet_name='REPORT_RAW', header=10)
today = pd.read_excel('../data/inzi/08 SEP/07-09-2021_IN-OUT DAILY REPORT.xlsb', sheet_name='REPORT-RAW', header=8)


result = pd.concat([previous, today])

result.rename(columns={
    'Unnamed: 1': 'Material'
}, inplace=True)

result['temp'] = 0

result = result[['Material', 'temp']]

result.drop_duplicates(inplace=True, keep='first')

# result = pd.merge(result, previous, how="left", on=['Material'], sort=False,
#                             indicator=False, validate=None)
# result = pd.merge(result, today, how="left", on=['Material'], sort=False,
#                             indicator=False, validate=None)
del result['temp']

In [22]:
del previous['Unnamed: 0']

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,321,344,401,623,720,201,261,321.1,344.1,401.1,601,609,721,Unnamed: 18,Unnamed: 19,321.2,323,327,344.2,401.2,623.1,720.1,201.1,261.1,321.3,323.1,327.1,344.3,401.3,601.1,609.1,721.1,Unnamed: 37,Unnamed: 38,262,...,401.6,623.3,720.3,201.3,261.3,344.7,401.7,601.3,609.3,712,721.3,Unnamed: 69,Unnamed: 70,343,344.8,401.8,623.4,720.4,201.4,261.4,343.1,344.9,401.9,601.4,609.4,721.4,Unnamed: 84,Unnamed: 85,321.6,401.10,623.5,720.5,201.5,261.5,321.7,401.11,601.5,609.5,721.5,Unnamed: 97
0,1,0103-004609,G,5730341.46,2550000.0,0.0,0.0,1200000.0,NaN,0.0,0.0,217442.8,0.0,0.0,1200000.0,0.0,0.0,0.0,8062898.66,0.0,0.0,0.0,0.0,0.0,850000.0,0.0,0.0,0.0,435425.0,0.0,0.0,0.0,0.0,850000.0,0.0,0.0,0.0,7627473.66,0.0,0.0,...,450000.0,0.0,0.0,0.0,100800.0,0.0,0.0,0.0,450000.0,0.0,0.0,7143573.66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7143573.66,0,0,0,0,0,0,0.0,0,0,0,0,0,7143573.66
1,2,0103-007368,G,7258704.00,2400000.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9658704.00,0.0,0.0,0.0,0.0,0.0,40000.0,0.0,0.0,0.0,33600.0,0.0,0.0,0.0,0.0,40000.0,0.0,0.0,0.0,9625104.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9625104.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9625104.00,0,0,0,0,0,0,0.0,0,0,0,0,0,9625104.00
2,3,0103-007683,G,774680.00,0.0,0.0,0.0,150000.0,NaN,0.0,0.0,113820.0,0.0,0.0,150000.0,0.0,0.0,0.0,660860.00,0.0,0.0,0.0,0.0,0.0,150000.0,0.0,0.0,0.0,196380.0,0.0,0.0,0.0,0.0,150000.0,0.0,0.0,0.0,464480.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,446030.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,446030.00,0,0,0,0,0,0,0.0,0,0,0,0,0,446030.00
3,4,0103-009882,G,8764600.00,850000.0,0.0,0.0,100000.0,NaN,0.0,0.0,100000.0,0.0,0.0,100000.0,0.0,0.0,0.0,9514600.00,0.0,0.0,0.0,0.0,0.0,125000.0,0.0,0.0,0.0,100900.0,0.0,0.0,0.0,0.0,125000.0,0.0,0.0,0.0,9413700.00,0.0,0.0,...,200000.0,0.0,0.0,0.0,103400.0,0.0,0.0,0.0,200000.0,0.0,0.0,9304700.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9304700.00,0,0,250000,0,0,0,111900.0,0,250000,0,0,0,9192800.00
4,5,0103-010237,G,2909545.00,0.0,0.0,0.0,990000.0,NaN,0.0,0.0,842640.0,0.0,0.0,990000.0,0.0,0.0,0.0,2066905.00,0.0,0.0,0.0,0.0,0.0,700000.0,0.0,0.0,0.0,490770.0,0.0,0.0,0.0,0.0,700000.0,0.0,0.0,0.0,1576135.00,0.0,0.0,...,200000.0,0.0,0.0,0.0,256860.0,0.0,0.0,0.0,200000.0,0.0,0.0,1140295.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1140295.00,1800000,0,400000,0,0,0,457380.0,0,400000,0,0,0,2482915.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634,635,BN96-53506U,PC,2340.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2340.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2340.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2340.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2340.00,900,0,0,0,0,0,0.0,0,0,0,0,0,3240.00
635,636,BN96-53692A,PC,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,1,0,0,0,0,0,0.0,0,0,0,0,0,1.00
636,637,BN96-53693A,PC,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,1,0,0,0,0,0,0.0,0,0,0,0,0,1.00
637,638,BN96-54014A,PC,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,...,NaN,N

In [23]:
today

,Unnamed: 0,Unnamed: 1,Unnamed: 2,101,102,Unnamed: 5,321,344,401,623,720,201,261,321.1,344.1,401.1,601,609,721,IN,OUT
0,1,0103-004609,G,0,0,0,0,0,100000,0,0,0,28700.0,0,0,100000,0,0,350000,0,378700.0
1,2,0103-009882,G,0,0,0,0,0,350000,0,0,0,52200.0,0,0,350000,0,0,0,0,52200.0
2,3,0103-010237,G,0,0,0,0,0,300000,0,0,0,297000.0,0,0,300000,0,0,0,0,297000.0
3,4,0103-011270,G,0,0,0,0,0,500000,0,0,0,73800.0,0,0,500000,0,0,0,0,73800.0
4,5,0103-011284,G,0,0,0,0,0,400000,0,0,0,167940.0,0,0,400000,0,0,0,0,167940.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363,364,HL96-52597A,PC,17502,0,17502,0,0,0,0,0,0,446.0,0,0,0,0,0,0,17502,446.0
364,365,HS02-00156A,PC,800,0,800,0,0,0,0,0,0,0.0,0,0,0,0,0,0,800,0.0
365,366,IZ01-000146,PC,2456,0,2456,0,0,0,0,0,0,0.0,0,0,0,0,0,0,2456,0.0
366,367,IZ01-000180,KG,7214,0,7214,0,0,0,0,0,0,0.0,0,0,0,0,0,0,7214,0.0
